In [34]:
import json
import numpy as np
import os
import pandas as pd
import requests
import time
from xml.etree import ElementTree as ET

In [45]:
# Lines of code used for checking code:

# response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?id=13')
# tree = ET.fromstring(response.content)
# List of all the elements
#[elem.tag for elem in tree.iter()]
# See the whole text in XML format
#print(ET.tostring(tree, encoding='utf8').decode('utf8'))

## Code to pull boardgame info

In [35]:
# Creating a function to get the boardgame data from each page

def boardgame_info(tree):
    
    id_num = int(tree[0].attrib['id'])
    boardgamecategory = []
    boardgamemechanic = []
    boardgamedesigner = []
    
    for i in range(len(tree[0])):
        
        if tree[0][i].tag == 'name':
            if tree[0][i].attrib['type'] == 'primary':
                boardgame_name = tree[0][i].attrib['value']
                
        elif tree[0][i].tag == 'yearpublished':
            year_publ = int(tree[0][i].attrib['value'])
            
        elif tree[0][i].tag == 'minplayers':
            minplayers = int(tree[0][i].attrib['value'])
            
        elif tree[0][i].tag == 'maxplayers':
            maxplayers = int(tree[0][i].attrib['value'])   
            
        elif tree[0][i].tag == 'playingtime':
            playingtime = int(tree[0][i].attrib['value']) 
            
        elif tree[0][i].tag == 'minplaytime':
            minplaytime = int(tree[0][i].attrib['value']) 
            
        elif tree[0][i].tag == 'maxplaytime':
            maxplaytime = int(tree[0][i].attrib['value'])
            
        elif tree[0][i].tag == 'link':
            
            if tree[0][i].attrib['type'] == 'boardgamecategory':
                boardgamecategory.append(tree[0][i].attrib['value'])
                
            elif tree[0][i].attrib['type'] == 'boardgamemechanic':
                boardgamemechanic.append(tree[0][i].attrib['value'])
                
            elif tree[0][i].attrib['type'] == 'boardgamedesigner':
                boardgamedesigner.append(tree[0][i].attrib['value'])
     
    keys = ['game ID#', 'boardgame', 'year published', 'min players', 'max players', 'playing time', 'min playing time', 'max playing time', 'category', 'boardgame mechanic', 'boardgamedesigner']
    values = [id_num, boardgame_name, year_publ, minplayers, maxplayers, playingtime, minplaytime, maxplaytime, boardgamecategory, boardgamemechanic, boardgamedesigner]
    dictionary = dict(zip(keys, values))

    return dictionary           

## Code to pull boardgame ratings

In [46]:
# Lines of code used for checking code:

# ratings_response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?id=13&ratingcomments=1&page=1')
# ratings_tree = ET.fromstring(ratings_response.content)

In [36]:
# Creating a function to get the boardgame ratings

def get_ratings_per_page(ratings_tree):
        
    # Figuring out the number of ratings pages
    for i in range(len(ratings_tree[0])):
        if ratings_tree[0][i].tag == 'comments':
            ratings_info = i
            num_ratings = float(ratings_tree[0][i].attrib['totalitems'])
    #print(ratings_info)

    all_local_ratings = []
    for i in range(len(ratings_tree[0][ratings_info])):

        username = ratings_tree[0][ratings_info][i].attrib['username']
        rating = float(ratings_tree[0][ratings_info][i].attrib['rating'])     

        ratings_keys = ['username', 'rating']
        ratings_values = [username, rating]
        ratings_dictionary = dict(zip(ratings_keys, ratings_values))
        
        all_local_ratings.append(ratings_dictionary)
        
        #print(ratings_dictionary)
    return all_local_ratings

In [37]:
def get_all_ratings(ID):
    
    site = 'https://www.boardgamegeek.com/xmlapi2/thing?id='
    page_num = ID   
    ratings_site = '&ratingcomments=1&page='
    ratings_page_num_1 = 1
       
    ratings_website = site + str(page_num) + ratings_site + str(ratings_page_num_1)
    ratings_response = requests.get(ratings_website)
    ratings_tree = ET.fromstring(ratings_response.content)
        
   
    # Figuring out the number of ratings pages
    num_ratings = 0
    for j in range(len(ratings_tree[0])):
        if ratings_tree[0][j].tag == 'comments':
            num_ratings = float(ratings_tree[0][j].attrib['totalitems'])
    #print(num_ratings)
    
    num_pages = round(num_ratings/100)
    if num_pages < num_ratings/100:
        num_pages += 1
    #print(num_pages)

    all_ratings = []
    for k in range(num_pages):
        ratings_website2 = site + str(page_num) + ratings_site + str(k+1)
        ratings_response2 = requests.get(ratings_website2)
        ratings_tree2 = ET.fromstring(ratings_response2.content)
        #print(k, ratings_website2)
        all_ratings.append(get_ratings_per_page(ratings_tree2))
        time.sleep(2)        
    return all_ratings

## Pulling all info and ratings of multiple games

In [38]:
def get_all_game_info(ID):
    
    # boardgame websites
    site = 'https://www.boardgamegeek.com/xmlapi2/thing?id='
    page_num = ID
    website = site+str(page_num)        
    response = requests.get(website)
    tree = ET.fromstring(response.content)
       
    ratings_site = '&ratingcomments=1&page='
    ratings_page_num_1 = 1
       
    ratings_website = site + str(page_num) + ratings_site + str(ratings_page_num_1)
    ratings_response = requests.get(ratings_website)
    ratings_tree = ET.fromstring(ratings_response.content) 

    bg_info = boardgame_info(tree)
    bg_ratings = get_all_ratings(ID)
    bg_info['ratings'] = bg_ratings
    time.sleep(5)         

    
    return bg_info

## Pulling all info from BoardGameGeek's API

In [ ]:
for ID in range(1,50001):
    site = 'https://www.boardgamegeek.com/xmlapi2/thing?id='
    page_num = ID
    website = site+str(page_num)        
    response = requests.get(website)
    tree = ET.fromstring(response.content)

    if len(tree) > 0 and tree[0].attrib['type'] == 'boardgame':
        ind_bg = get_all_game_info(ID)
        #print(ID)
        with open('ind_bg_{0}.json'.format(ID), 'w') as json_file:
            json.dump(ind_bg, json_file)
    
    time.sleep(5)

In [27]:
# updating the path
path = 'C:/Users/Ariel/Desktop/BGG_JSON_files/'
os.chdir(path)

# Loop through all the json files in the forlder
ind_bg = []
for i in range(1,49984):
    
    if os.path.exists('ind_bg_{0}.json'.format(i)) == True:
        
        #open the file
        with open('ind_bg_{0}.json'.format(i)) as f:
          ind_bg_json = json.load(f)
        #ind_bg = pd.DataFrame.from_dict(ind_bg_json, orient = 'index').T
        ind_bg.append(ind_bg_json)

In [33]:
# Path to save board_game_df.csv to 
path2 = 'C:/Users/Ariel/Desktop/01 DATA SCIENCE/Springboard/Capstone 1/'

# saving as csv
board_game_df.to_csv (path2+'board_game_df.csv', index = None, header=True) 

In [41]:
# Updating the path to get board_game_df.csv
file = 'C:/Users/Ariel/Desktop/01 DATA SCIENCE/Springboard/Capstone 1/'

# Loading board_game_df.csv
board_game_df = pd.read_csv(file+'board_game_df.csv', encoding = 'unicode_escape')

In [42]:
board_game_df.head()

,boardgame,boardgame mechanic,boardgamedesigner,category,game ID#,max players,max playing time,min players,min playing time,playing time,ratings,year published
0,Die Macher,"['Area Control / Area Influence', 'Auction/Bid...",['Karl-Heinz Schmiel'],"['Economic', 'Negotiation', 'Political']",1,5,240,3,240,240,"[[{'username': 'jackcres', 'rating': 10.0}, {'...",1986
1,Dragonmaster,['Trick-taking'],"['G. W. ""Jerry"" D'Arcey']","['Card Game', 'Fantasy']",2,4,30,3,30,30,"[[{'username': 'bard', 'rating': 10.0}, {'user...",1981
2,Samurai,"['Area Control / Area Influence', 'Hand Manage...",['Reiner Knizia'],"['Abstract Strategy', 'Medieval']",3,4,60,2,30,60,"[[{'username': 'Paul Gower', 'rating': 10.0}, ...",1998
3,Tal der Könige,"['Action Point Allowance System', 'Area Contro...",['Christian Beierer'],['Ancient'],4,4,60,2,60,60,"[[{'username': 'rdsmith', 'rating': 10.0}, {'u...",1992
4,Acquire,"['Hand Management', 'Stock Holding', 'Tile Pla...",['Sid Sackson'],['Economic'],5,6,90,2,90,90,"[[{'username': 'Blaster', 'rating': 10.0}, {'u...",1964
